In [2]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler





airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv')

flights['DATE_DAY'] = pd.to_datetime(flights[['YEAR', 'MONTH', 'DAY']])

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_67356/2975684598.py:21: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv('flights.csv')


In [3]:
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.DATE_DAY.dt.year
    data['month'] = data.DATE_DAY.dt.month
    data['day'] = data.DATE_DAY.dt.day
    data['dayofweek'] = data.DATE_DAY.dt.dayofweek
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['ARRIVAL_DELAY'].shift(lag)

In [10]:
start_airport = np.random.choice(flights['ORIGIN_AIRPORT'].unique()) 


df = flights.query("ORIGIN_AIRPORT == '{}'".format(start_airport))
total = pd.DataFrame()

for d_airport in df['DESTINATION_AIRPORT'].unique():
    d_df = df.query("DESTINATION_AIRPORT == '{}'".format(d_airport))
    
    
    
    Q1 = d_df['ARRIVAL_DELAY'].quantile(0.25)
    Q3 = d_df['ARRIVAL_DELAY'].quantile(0.75)
    IQR = Q3 - Q1
    a = d_df[(d_df['ARRIVAL_DELAY'] < Q1-1.5*IQR ) | (d_df['ARRIVAL_DELAY'] > Q3+1.5*IQR)]['ARRIVAL_DELAY'].reset_index()
    a = list(a.ARRIVAL_DELAY)
    d_df = d_df.query("ARRIVAL_DELAY != @a")



    try:    

        d_df = d_df.groupby('DATE_DAY')['ARRIVAL_DELAY'].sum().reset_index()
        make_features(d_df,21,7)
        d_df.dropna(inplace=True)
        d_df.set_index('DATE_DAY', inplace=True)



        X_train,X_test, y_train, y_test = train_test_split(d_df.drop('ARRIVAL_DELAY',axis=1),
                                                        d_df.ARRIVAL_DELAY,
                                                        shuffle = False,
                                                        test_size = 0.25)

    
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        pred = lr.predict(X_test)



    except Exception as e:
        continue


    t = {'name':'{}'.format(d_airport), 'RMSE':round(np.sqrt(mean_squared_error(y_test, pred)), 2), 'ARRIVAL_DELAY':round(y_test.mean(), 2)}
    total = total.append(t, ignore_index=True)


    #print(d_airport,'Ошибка на тесте', (mean_squared_error(y_test, pred))**0.5, y_test.mean())


total = total.sort_values(by=['RMSE','ARRIVAL_DELAY'],ascending=[True, True])
print(total)
total = total.head(4)
t = {'name':'{}'.format(start_airport), 'RMSE':0, 'ARRIVAL_DELAY':total.ARRIVAL_DELAY.abs().mean()*2}
total = total.append(t, ignore_index=True)

map_air = airports.merge(total, how='right', left_on='IATA_CODE', right_on='name')


fig = px.scatter_mapbox(total,
                        lon = map_air.LONGITUDE,
                        lat = map_air.LATITUDE,
                        zoom = 3,
                        color = map_air.ARRIVAL_DELAY.abs(),
                        size = map_air.ARRIVAL_DELAY.abs(),
                        width = 800,
                        height = 600,
)
fig.update_layout(mapbox_style='open-street-map')
fig.show()


  name   RMSE  ARRIVAL_DELAY
2  DEN  19.56          -5.26
0  PHX  19.86          -6.74
1  DFW  28.06         -20.09


/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_67356/1248384212.py:47: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_67356/1248384212.py:47: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_67356/1248384212.py:47: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_67356/1248384212.py:57: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

